In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
run = 3

In [3]:
import os

if os.path.exists(f"adversarials\\Final\\run{run:02}\\res.csv"):
    raise ValueError("Unable to run this run")

In [4]:
import torch.nn as nn
import torch
from matplotlib import pyplot as plt
import torchvision

In [5]:
from PIL import Image
import numpy as np

In [6]:
from metrics import Metric, WassersteinApproximation, StructuralDissimilarity, L2Metric, LpMetric

In [7]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.seq = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.2),
            nn.Conv2d(32, 64, 3),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(1024, 200),
            nn.ReLU(),
            nn.Linear(200, 10),
            nn.Softmax(dim=1)
        )
    
    def forward(self, x):
        return self.seq(x)

In [8]:
model = torch.load('models\\model_v1.model')

train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST(
        'mnist',
        train=True,
        download=True,
        transform=torchvision.transforms.ToTensor()
    ),
    batch_size=60,
    shuffle=False
)

In [9]:
def get_benign_examples(model, dataloader, count, run_num):
    counter = 0
    benign_examples = torch.zeros(count, 1, dataloader.dataset[0][0].shape[1], dataloader.dataset[0][0].shape[2])
    benign_labels = torch.zeros(count)
    for i, el in enumerate(dataloader):
        examples, labels = el
        if i < (run_num - 1) * 100:
            continue
        preds = model(examples)
        match = (torch.argmax(preds, dim=1) == labels)
        for idx, foo in enumerate(match):
            if foo:
                benign_examples[counter] = examples[idx]
                benign_labels[counter] = labels[idx]
                counter += 1
            if counter >= count:
                break
        if counter >= count:
            break
    return benign_examples, benign_labels

In [10]:
batch = 100

benign, labels = get_benign_examples(model, train_loader, batch, run)


In [11]:
def cw_batch(model: nn.Module, benign_examples: torch.Tensor, labels: torch.Tensor, c_lambda: float, metric: Metric, special_init = False) -> torch.Tensor:
    if special_init:
        adversarial_examples = benign_examples
    else:
        adversarial_examples = 0.5 * torch.ones(benign_examples.shape) + 0.3 * (2 * torch.rand(benign_examples.shape) - 1)
    loss_fn = nn.CrossEntropyLoss(reduction='sum')
    step_size = 1e-2
    for i in range(100):
        adversarial_examples.requires_grad = True
        if adversarial_examples.grad is not None:
            adversarial_examples.grad.zero_()
        benign_examples.requires_grad = True
        if benign_examples.grad is not None:
            benign_examples.grad.zero_()
        metrics = metric(benign_examples, adversarial_examples)
        
        loss = metrics[metrics == metrics].sum() - c_lambda * loss_fn(model(adversarial_examples), torch.tensor(labels, dtype=torch.long))
        loss.backward()
        adversarial_examples = (adversarial_examples - step_size * adversarial_examples.grad.apply_(lambda x: 1 if x >= 0 else -1)).detach()
        adversarial_examples[adversarial_examples < 0] = 0
        adversarial_examples[adversarial_examples > 1] = 1
        if (i + 1) % 10 == 0:
            print(f"::DONE {i+1} iters of CW")
    return adversarial_examples

In [12]:
lambdas = [1000, 100, 10, 1, 0.1, 0.01, 0.001]

metrics = []

# Wasserstein
for regularization in [5, 10, 15]:
    metrics.append(
        {
            'metric': WassersteinApproximation(regularization=regularization, iterations=150),
            'name': f'WassersteinAproximation-reg={regularization}'
        }
    )

In [14]:
l2 = L2Metric()

res = ["Metric,Lambda,Success,MetricDistance,L2Distance,Run,Example"]

for metric in metrics:
    for lambd in lambdas:
        adv = cw_batch(model, benign, labels, lambd, metric['metric'])
        metric['adv'] = adv
        metric['success'] = torch.argmax(model(adv), dim=1) != labels
        metric['dist'] = metric['metric'](benign, adv)
        metric['L2_dist'] = l2(benign, adv)
        for i, example in enumerate(adv):
            res.append(f"{metric['name']},{lambd},{metric['success'][i]},{metric['dist'][i]},{metric['L2_dist'][i]},{run+1},{i+1}")
        print(f':DONE lambda = {lambd}')
    print(f"DONE metric {metric['name']}")
    
with open(f"adversarials\\Final\\run{run:02}\\res.csv", 'w') as f:
    f.writelines(res)

C:\Users\stani\AppData\Local\Temp\ipykernel_14512\3488933475.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = metrics[metrics == metrics].sum() - c_lambda * loss_fn(model(adversarial_examples), torch.tensor(labels, dtype=torch.long))


::DONE 10 iters of CW
::DONE 20 iters of CW
::DONE 30 iters of CW
::DONE 40 iters of CW
::DONE 50 iters of CW
::DONE 60 iters of CW
::DONE 70 iters of CW
::DONE 80 iters of CW
::DONE 90 iters of CW
::DONE 100 iters of CW
:DONE lambda = 1000
::DONE 10 iters of CW
::DONE 20 iters of CW
::DONE 30 iters of CW
::DONE 40 iters of CW
::DONE 50 iters of CW
::DONE 60 iters of CW
::DONE 70 iters of CW
::DONE 80 iters of CW
::DONE 90 iters of CW
::DONE 100 iters of CW
:DONE lambda = 100
::DONE 10 iters of CW
::DONE 20 iters of CW
::DONE 30 iters of CW
::DONE 40 iters of CW
::DONE 50 iters of CW
::DONE 60 iters of CW
::DONE 70 iters of CW
::DONE 80 iters of CW
::DONE 90 iters of CW
::DONE 100 iters of CW
:DONE lambda = 10
::DONE 10 iters of CW
::DONE 20 iters of CW
::DONE 30 iters of CW
::DONE 40 iters of CW
::DONE 50 iters of CW
::DONE 60 iters of CW
::DONE 70 iters of CW
::DONE 80 iters of CW
::DONE 90 iters of CW
::DONE 100 iters of CW
:DONE lambda = 1
::DONE 10 iters of CW
::DONE 20 iters of C

In [15]:
with open(f"adversarials\\Final\\run{run:02}\\res.csv", 'w') as f:
    f.writelines([line + '\n' for line in res])